In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
TempData = pd.read_csv('/content/drive/MyDrive/MonthWiseMarketArrivals_Clean (1).csv')
TempData.head()

FileNotFoundError: ignored

In [ ]:
TempData.columns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TempData.Country.unique()

In [ ]:
TempData.City.unique()

In [ ]:
TempData.shape

In [ ]:
#Data Cleaning
TempData = TempData.dropna()

In [ ]:
df_Denamrk = TempData[TempData.Country == "Denmark"]

In [ ]:
df_Denamrk.index = pd.to_datetime(df_Denamrk.dt)

In [ ]:
df_Denamrk = df_Denamrk.drop(['dt','AverageTemperatureUncertainty'], axis=1)

In [ ]:
df_Denamrk.describe()

Create Time Series

In [ ]:
ts = df_Denamrk['AverageTemperature']
ts

In [ ]:
#Data Exploration
plt.figure(figsize=(16,7))
fig = plt.figure(1)
ax1 = fig.add_subplot(111)
ax1.set_xlabel('Time Frame')
ax1.set_ylabel('Average Temp Recorded')
ax1.plot(ts)

In [ ]:
# Perform Augmented Dickey-Fuller test:
# ADF Test - nul hypothesis - non-stationary - if p-value < 5% reject null hypothesis
adfuller_result = adfuller(ts, autolag='AIC')


In [ ]:
print(f'ADF Statistic: {adfuller_result[0]}')

In [ ]:
print(f'p-value: {adfuller_result[1]}')

In [ ]:
for key, value in adfuller_result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

In [ ]:
#First model
newmodel = ARMA(ts,order=(1,1))

In [ ]:
results = newmodel.fit()

In [ ]:
predictions = results.predict('01/01/1990', '01/01/1991')

In [ ]:
actuals = df_Denamrk['01/01/1990': '01/01/1991']['AverageTemperature'][0:13]

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actuals[0:13], predictions)
print('MAE: %f' % mae)

In [ ]:
import itertools
i = j = range(0, 4)
ij = itertools.product(i,j)
for parameters in ij:
    try:
        mod = ARMA(ts,order=parameters)
        results = mod.fit()
        print('ARMA{} - AIC:{}'.format(parameters, results.aic))
    except:
        continue

In [ ]:
predictions = results.predict('01/01/1990', '01/01/1991')

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actuals[0:13], predictions)
print('MAE: %f' % mae)